In [22]:
using BenchmarkTools
using LinearAlgebra
using StaticArrays

## Vecchia funzione

In [23]:
function t(args...)
	d = length(args)
	mat = Matrix{Float64}(LinearAlgebra.I, d+1, d+1)
	for k in range(1, length=d)
        	mat[k,d+1]=args[k]
	end
	return mat
end

t (generic function with 1 method)

In [24]:
@btime t(-0.5,-0.5)

  79.325 ns (1 allocation: 160 bytes)


3×3 Array{Float64,2}:
 1.0  0.0  -0.5
 0.0  1.0  -0.5
 0.0  0.0   1.0

In [25]:
@code_warntype t(-0.5,-0.5)

Variables
  #self#::Core.Compiler.Const(t, false)
  args::Tuple{Float64,Float64}
  d::Int64
  mat::Array{Float64,2}
  @_5::Union{Nothing, Tuple{Int64,Int64}}
  k::Int64

Body::Array{Float64,2}
1 ─       (d = Main.length(args))
│   %2  = Core.apply_type(Main.Matrix, Main.Float64)::Core.Compiler.Const(Array{Float64,2}, false)
│   %3  = LinearAlgebra.I::Core.Compiler.Const(UniformScaling{Bool}(true), false)
│   %4  = (d::Core.Compiler.Const(2, false) + 1)::Core.Compiler.Const(3, false)
│   %5  = (d::Core.Compiler.Const(2, false) + 1)::Core.Compiler.Const(3, false)
│         (mat = (%2)(%3, %4, %5))
│   %7  = (:length,)::Core.Compiler.Const((:length,), false)
│   %8  = Core.apply_type(Core.NamedTuple, %7)::Core.Compiler.Const(NamedTuple{(:length,),T} where T<:Tuple, false)
│   %9  = Core.tuple(d::Core.Compiler.Const(2, false))::Core.Compiler.Const((2,), false)
│   %10 = (%8)(%9)::Core.Compiler.Const((length = 2,), false)
│   %11 = Core.kwfunc(Main.range)::Core.Compiler.Const(Base.var"#rang

In [26]:
@code_llvm t(-0.5,-0.5)


;  @ In[23]:1 within `t'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_t_2088(double, double) #0 {
pass:
  %gcframe = alloca %jl_value_t*, i32 3, align 16
  %2 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %2, i8 0, i32 24, i1 false)
  %3 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #7
;  @ In[23]:3 within `t'
; ┌ @ C:\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.5\LinearAlgebra\src\uniformscaling.jl:433 within `Array'
   %4 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
   %5 = bitcast %jl_value_t** %4 to i64*
   store i64 4, i64* %5
   %6 = getelementptr %jl_value_t**, %jl_value_t*** %3, i32 0
   %7 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
   %8 = bitcast %jl_value_t** %7 to %jl_value_t***
   %9 = load %jl_value_t**, %jl_value_t*** %6
   store %jl_value_t** %9, %jl_value_t*** %8
   %10 = bitcast %jl_value_t*** %6 to %jl_value_t***
   store %jl_value_t**

In [27]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 
  memory estimate:  160 bytes
  allocs estimate:  1
  --------------
  minimum time:     75.819 ns (0.00% GC)
  median time:      124.071 ns (0.00% GC)
  mean time:        190.057 ns (1.87% GC)
  maximum time:     9.229 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     914

## Modifiche

"""\
t(args::Array{Float64,1}...)::MMatrix\
Genera e ritorna una matrice statica “mat” di trasformazione affine(nello specifico di traslazione) in coordinate omogenee. Questa matrice ha “d + 1” righe e “d + 1” colonne, dove “d” è il numero di parametri nell'array “args” passato come argomento.

Esempi
```julia
julia> t(1,2)			# 2D translation
# return
3×3 Array{Float64,2}:
 1.0  0.0  1.0
 0.0  1.0  2.0
 0.0  0.0  1.0
julia> Lar.t(1.,2,3)		# 3D translation
# return
4×4 Array{Float64,2}:
 1.0  0.0  0.0  1.0
 0.0  1.0  0.0  2.0
 0.0  0.0  1.0  3.0
 0.0  0.0  0.0  1.0
```
"""

In [28]:
function t(args...)
	d = length(args)
	mat = MMatrix{d+1,d+1,Float64}(1I) 
	@inbounds for k in range(1, length=d)
        	mat[k,d+1]=args[k]
	end
	return mat
end

t (generic function with 1 method)

In [29]:
@btime t(-0.5,-0.5)

  15.130 ns (1 allocation: 80 bytes)


3×3 MArray{Tuple{3,3},Float64,2,9} with indices SOneTo(3)×SOneTo(3):
 1.0  0.0  -0.5
 0.0  1.0  -0.5
 0.0  0.0   1.0

In [30]:
@code_llvm t(-0.5,-0.5)


;  @ In[28]:1 within `t'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_t_2238(double, double) #0 {
top:
  %2 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #6
;  @ In[28]:3 within `t'
; ┌ @ C:\Users\omarm\.julia\packages\StaticArrays\LJQEe\src\linalg.jl:118 within `StaticArray'
; │┌ @ C:\Users\omarm\.julia\packages\StaticArrays\LJQEe\src\MMatrix.jl:40 within `MArray'
; ││┌ @ C:\Users\omarm\.julia\packages\StaticArrays\LJQEe\src\MMatrix.jl:43 within `macro expansion'
; │││┌ @ C:\Users\omarm\.julia\packages\StaticArrays\LJQEe\src\MArray.jl:30 within `MArray'
      %3 = bitcast %jl_value_t*** %2 to i8*
      %4 = call noalias nonnull %jl_value_t* @jl_gc_pool_alloc(i8* %3, i32 1496, i32 80) #3
      %5 = bitcast %jl_value_t* %4 to %jl_value_t**
      %6 = getelementptr %jl_value_t*, %jl_value_t** %5, i64 -1
      store %jl_value_t* inttoptr (i64 349195376 to %jl_value_t*), %jl_value_t** %6
      %7 = bitcast %jl_value_t* %4 to [9 x double]*
      %.

In [31]:
@benchmark t(-0.5,-0.5)

BenchmarkTools.Trial: 
  memory estimate:  80 bytes
  allocs estimate:  1
  --------------
  minimum time:     15.130 ns (0.00% GC)
  median time:      18.236 ns (0.00% GC)
  mean time:        31.662 ns (15.16% GC)
  maximum time:     4.318 μs (97.76% GC)
  --------------
  samples:          10000
  evals/sample:     998